In [89]:
import math
import random
import pygame
import tkinter as tk
from tkinter import messagebox

class options:
    
    # Theese are tweakable. Adjust to fit your needs.
    width = 1000
    rows = 50 
    fps_limit = 20 
    Swidth = width // rows

class colors:
        
    gold = 255, 215, 0
    silver = 192, 192, 192
    black = 0, 0, 0
    green = 0, 128, 0
    red = 255, 0, 0
    cyan = 0, 255, 255
    
class square:
    
    def __init__(self, coords, color):
        
        self.coords = coords
        self.color = color
        self.vx = 0
        self.vy = -1

    def move(self, vx, vy):
        
        self.vx = vx
        self.vy = vy
        self.coords = (self.coords[0] + self.vx, self.coords[1] + self.vy)
 
    def draw(self, surface, eyes=False):
            
        SW = options.Swidth
        pygame.draw.rect(surface, self.color, (self.coords[0]*SW+1, self.coords[1]*SW+1, SW-1, SW-1))
        
        if eyes:
            centre = SW//2
            radius = 3
            circleMiddle = (self.coords[0]*SW+centre-radius, self.coords[1]*SW+8)
            circleMiddle2 = (self.coords[0]*SW+SW-radius*2, self.coords[1]*SW+8)
            pygame.draw.circle(surface, colors.black, circleMiddle, radius)
            pygame.draw.circle(surface, colors.black, circleMiddle2, radius)

class wall:
        
    def __init__(self):
        
        self.w = []
        self.R = options.rows
        
    def build(self):
        
        B = colors.black
    
        for j in range (self.R):
            self.w.append(square((j, 0), color=B))
            self.w.append(square((j, self.R - 1), color=B))
        
            if j != 0 and j != self.R - 1:
                self.w.append(square((0, j), color=B))
                self.w.append(square((self.R - 1, j), color=B))
                
            if 1.5 * self.R // 5 <= j < 3.5 * self.R // 5:
                self.w.append(square((self.R // 3, j), color=B))
                self.w.append(square((2 * self.R // 3, j), color=B))
                
    
    def draw(self, surface):
        
        for j in range (4*self.R+4*self.R//5-4):
            self.w[j].draw(surface)
    
class food:
    
    def __init__(self):
        
        self.R = options.rows
        self.numy = 0, 0
        self.yuck = 0, 0
        
    def randomSnack(self, snake1, snake2, wall, num=1):

        C = colors.cyan
        R = colors.red
        s1 = snake1.body
        s2 = snake2.body
    
        while True:
            x = random.randrange(1, self.R-1)
            y = random.randrange(1, self.R-1)
            
            condition1 = len(list(filter(lambda z:z.coords == (x, y), s1)))
            condition2 = len(list(filter(lambda z:z.coords == (x, y), s2)))
            condition3 = len(list(filter(lambda z:z.coords == (x, y), wall)))
            
            if condition1 > 0 or condition2 > 0 or condition3 > 0:
                continue
                
            else:
                
                if num == 1 and self.yuck != (x, y):
                    self.numy = x, y
                    self.nsquare = square((x, y), color=R)
                    break

                elif num == 0 and self.numy != (x, y):
                    self.yuck = x, y
                    self.ysquare = square((x, y), color=C)
                    break
                
                else: 
                    
                    continue
    
    def draw(self, surface):
        
        self.nsquare.draw(surface=surface)
        self.ysquare.draw(surface=surface)
    
    
    
class snake:
    
    def __init__(self, coords, color):
        
        self.color = color
        self.start = coords
        self.body = []
        self.turns = {}
        self.head = square(self.start, self.color)
        self.body.append(self.head)
        self.vx = 0
        self.vy = 0
        self.RIP = 1
 
    def move_arrows(self):
        
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                pygame.quit()
 
            self.keys = pygame.key.get_pressed()
  
            for key in self.keys:
                if self.keys[pygame.K_LEFT] and self.vx != 1:
                    self.vx = -1
                    self.vy = 0
                    self.turns[self.head.coords[:]] = [self.vx, self.vy]
 
                elif self.keys[pygame.K_RIGHT] and self.vx != -1:
                    self.vx = 1
                    self.vy = 0
                    self.turns[self.head.coords[:]] = [self.vx, self.vy]
 
                elif self.keys[pygame.K_UP] and self.vy != 1:
                    self.vx = 0
                    self.vy = -1
                    self.turns[self.head.coords[:]] = [self.vx, self.vy]
 
                elif self.keys[pygame.K_DOWN] and self.vy != -1:
                    self.vx = 0
                    self.vy = 1
                    self.turns[self.head.coords[:]] = [self.vx, self.vy]
 
        for i, c in enumerate(self.body):
            p = c.coords[:]
            
            if p in self.turns:
                turn = self.turns[p]
                c.move(turn[0],turn[1])
                
                if i == len(self.body)-1:
                    self.turns.pop(p)
                
            else:
                c.move(c.vx, c.vy)
                
    def move_wasd(self):
        
        self.keys = pygame.key.get_pressed()
  
        for key in self.keys:
            if self.keys[pygame.K_a] and self.vx != 1:
                self.vx = -1
                self.vy = 0
                self.turns[self.head.coords[:]] = [self.vx, self.vy]

            elif self.keys[pygame.K_d] and self.vx != -1:
                self.vx = 1
                self.vy = 0
                self.turns[self.head.coords[:]] = [self.vx, self.vy]

            elif self.keys[pygame.K_w] and self.vy != 1:
                self.vx = 0
                self.vy = -1
                self.turns[self.head.coords[:]] = [self.vx, self.vy]

            elif self.keys[pygame.K_s] and self.vy != -1:
                self.vx = 0
                self.vy = 1
                self.turns[self.head.coords[:]] = [self.vx, self.vy]
 
        for i, c in enumerate(self.body):
            p = c.coords[:]
            
            if p in self.turns:
                turn = self.turns[p]
                c.move(turn[0],turn[1])
                
                if i == len(self.body)-1:
                    self.turns.pop(p)
                
            else:
                c.move(c.vx, c.vy)
       
 
    def reset(self):
        
        self.head = square(self.start, self.color)
        self.body = []
        self.body.append(self.head)
        self.turns = {}
        self.vx = 0
        self.vy = 1
 
 
    def addSquare(self):
        
        tail = self.body[-1]
        dx, dy = tail.vx, tail.vy
 
        if dx == 1 and dy == 0:
            self.body.append(square((tail.coords[0]-1,tail.coords[1]), self.color))
        elif dx == -1 and dy == 0:
            self.body.append(square((tail.coords[0]+1,tail.coords[1]), self.color))
        elif dx == 0 and dy == 1:
            self.body.append(square((tail.coords[0],tail.coords[1]-1), self.color))
        elif dx == 0 and dy == -1:
            self.body.append(square((tail.coords[0],tail.coords[1]+1), self.color))
 
        self.body[-1].vx = dx
        self.body[-1].vy = dy
        
    def removeSquare(self):
        if len(self.body) > 1:
            del self.body[-1]
        else:
            self.RIP = 0

    def draw(self, surface):
        
        for i, c in enumerate(self.body):
            if i == 0:
                c.draw(surface, eyes=True)
            else:
                c.draw(surface)

class game:
    
    def __init__(self):
        pass
    
    def redrawWindow(self, window):
        
        window.fill(colors.green)
        self.W.draw(surface=window)
        self.s1.draw(window)
        self.s2.draw(window)
        self.F.draw(window)
        pygame.display.update()

    
    def message_box(self, subject, content):
        root = tk.Tk()
        root.attributes("-topmost", True)
        root.withdraw()
        messagebox.showinfo(subject, content)
        try:
            root.destroy()
        except:
            pass
 
 
    def play(self, two_players=False, render=True):
        
        r = options.rows
        screen_size = options.width, options.width
        self.W = wall()
        self.W.build()
        
        if two_players == True:
            self.s1 = snake((7*r//8, 7*r//8), colors.gold)
            self.s2 = snake((r//8, 7*r//8), colors.silver)
        else:
            self.s1 = snake((r//2, r//2), colors.gold)
            self.s2 = snake((-1, -1), colors.silver)
            
        self.F = food()
        self.F.randomSnack(self.s1, self.s2, self.W.w)
        self.F.randomSnack(self.s1, self.s2, self.W.w, num=0)
            
        if render == True:
            window = pygame.display.set_mode(screen_size)
            clock = pygame.time.Clock()
            #pygame.mixer.init()
            #pygame.mixer.music.load('C:/Users/Asas/Downloads/D.mp3') # Insert your favorite music here with path to it 
            #pygame.mixer.music.play(-1)                
        while True:
            
            if render == True:
                pygame.time.delay(50)
                clock.tick(options.fps_limit)
            
            if two_players == True:
                self.s1.move_arrows()
                self.s2.move_wasd()
                    
                if self.s1.body[0].coords == self.F.nsquare.coords:
                    self.s1.addSquare()
                    self.F.randomSnack(self.s1, self.s2, self.W.w)
                if self.s1.body[0].coords == self.F.ysquare.coords:
                    self.s1.removeSquare()
                    self.F.randomSnack(self.s1, self.s2, self.W.w, num=0) 
                if self.s2.body[0].coords == self.F.nsquare.coords:
                    self.s2.addSquare()
                    self.F.randomSnack(self.s1, self.s2, self.W.w)
                if self.s2.body[0].coords == self.F.ysquare.coords:
                    self.s2.removeSquare()
                    self.F.randomSnack(self.s1, self.s2, self.W.w, num=0) 
                
            else:
                self.s1.move_arrows()
                    
                if self.s1.body[0].coords == self.F.nsquare.coords:
                    self.s1.addSquare()
                    self.F.randomSnack(self.s1, self.s2, self.W.w)
                if self.s1.body[0].coords == self.F.ysquare.coords:
                    self.s1.removeSquare()
                    self.F.randomSnack(self.s1, self.s2, self.W.w, num=0) 
                
            if self.s1.body[0].coords in list(map(lambda z:z.coords, self.s1.body[1:])):
                if render == True:
                    message = 'Ouroborus! Player 1 bit his tail. \nPlayer 1 score: ' + str(len(self.s1.body)) + \
                    '\nPlayer 2 score: ' + str(len(self.s2.body))
                    self.message_box('Player 1 lost!', message)
                self.s1.reset()
                self.s2.reset()
                break
            elif self.s1.body[0].coords in list(map(lambda z:z.coords, self.W.w)):
                if render == True:
                    message = 'Tsk tsk tsk... Player 1 should mind the suroundings more. \nPlayer 1 score: ' + str(len(self.s1.body)) + \
                    '\nPlayer 2 score: ' + str(len(self.s2.body))
                    self.message_box('Player 1 lost!', message)
                self.s1.reset()
                self.s2.reset()
                break
            elif self.s1.body[0].coords in list(map(lambda z:z.coords, self.s2.body[1:])):
                if render == True:
                    message = 'Player 1 is now a snack to Player 2. \nPlayer 1 score: ' + str(len(self.s1.body)) + \
                    '\nPlayer 2 score: ' + str(len(self.s2.body))
                    self.message_box('Player 1 lost!', message)
                self.s1.reset()
                self.s2.reset()
                break

            if self.s2.body[0].coords in list(map(lambda z:z.coords, self.s2.body[1:])):
                if render == True:
                    message = 'Ouroborus! Player 2 bit his tail. \nPlayer 1 score: ' + str(len(self.s1.body)) + \
                    '\nPlayer 2 score: ' + str(len(self.s2.body))
                    self.message_box('Player 2 lost!', message)
                self.s1.reset()
                self.s2.reset()
                break
            elif self.s2.body[0].coords in list(map(lambda z:z.coords, self.W.w)):
                if render == True:
                    message = 'Tsk tsk tsk... Player 2 should mind the suroundings more. \nPlayer 1 score: ' + str(len(self.s1.body)) + \
                    '\nPlayer 2 score: ' + str(len(self.s2.body))
                    self.message_box('Player 2 lost!', message)
                self.s1.reset()
                self.s2.reset()
                break
            elif self.s2.body[0].coords in list(map(lambda z:z.coords, self.s1.body[1:])):
                if render == True:
                    message = 'Player 2 is now a snack to Player 1. \nPlayer 1 score: ' + str(len(self.s1.body)) + \
                    '\nPlayer 2 score: ' + str(len(self.s2.body))
                    self.message_box('Player 2 lost!', message)
                self.s1.reset()
                self.s2.reset()
                break
                
            if self.s1.body[0].coords == self.s2.body[0].coords:
                if render == True:
                    message = 'Head-on! That had to hurt... \nPlayer 1 score: ' + str(len(self.s1.body)) + \
                    '\nPlayer 2 score: ' + str(len(self.s2.body))
                    self.message_box('R.I.P. poor snakes', message)
                self.s1.reset()
                self.s2.reset()
                break
                
            if self.s1.RIP == 0:
                if render == True:
                    message = 'Player 1 should\'ve taken the red pill. \nPlayer 1 score: ' + str(len(self.s1.body)-1) + \
                    '\nPlayer 2 score: ' + str(len(self.s2.body))
                    self.message_box('Player 1 lost!', message)
                self.s1.reset()
                self.s2.reset()
                break
            if self.s2.RIP == 0:
                if render == True:
                    message = 'Player 2 should\'ve taken the red pill. \nPlayer 1 score: ' + str(len(self.s1.body)) + \
                    '\nPlayer 2 score: ' + str(len(self.s2.body)-1)
                    self.message_box('Player 2 lost!', message)
                self.s1.reset()
                self.s2.reset()
                break
                
            if render == True:
                self.redrawWindow(window)
                
        #pygame.mixer.music.stop
        pygame.quit()
        #sys.exit()
        
g = game()
g.play(two_players=True, render=True)